In [44]:
import pandas as pd
import numpy as np 
import os

In [56]:
db_dir = '/home/panastas/mount/PhD_stuff/PARALiA-GEMMex/silver1-build/silver1_2V100-install/Database/'
db_dir_mb = db_dir + 'microbenchmarks'
CHL_WORKERS = 2
CHL_MEMLOCS = CHL_WORKERS + 2
memloc_list = []
for i in range(0,CHL_MEMLOCS):
    memloc_list.append(i)

In [57]:
def translate_binary_to_unit_list(case_id):
    id_list = []
    for mask_offset in range(0, CHL_WORKERS+1):
        mask = 1 << mask_offset;
        if (case_id & mask):
            id_list.append(mask_offset);
    return id_list

def translate_unit_list_to_binary(list_input):
    case_id_out = 0; 
    for mask_offset in range(0, CHL_WORKERS+1):
        if mask_offset in list_input:
            case_id_out+= pow(2, mask_offset);
    return case_id_out;

def bench_bc_log_read(filename):
    if os.path.exists(filename):
        temp = pd.read_csv(filename, header=None, names = ['dest','dim1', 'dim2', 'elemSize','bw'], 
            dtype = {'dest': np.int64, 'dim1': np.int64, 'dim2': np.int64, 'elemSize': np.int64, 'bw': np.float64} )
        read_flag = 1
    else:
        temp = pd.DataFrame()
        read_flag = 0
    return read_flag, temp

def bench_bc_groupby_(df):
    result = df.groupby(['T','dev_num', 'dev_id_token', 'cachesize_limit','TransA','TransB',
        'alpha','beta','M','N','K','Asloc','Bsloc','Csloc', 'CsOutloc', 'flops', 'bytes', 'loc_str', 'imp', 'plot_order'], as_index=False, sort=True).mean()
    #print(result)
    return result

In [62]:
for case_id in range(1,2**CHL_WORKERS):
    print(case_id)
    active_unit_id_list = translate_binary_to_unit_list(case_id)
    print(active_unit_id_list)
    bench_bc = pd.DataFrame()
    cont_flag = 0
    for elem in memloc_list:
        bw_bench_bc_filename =  '%s/bw_bench_broadcast_2D_%d_%d.log' %(db_dir_mb, elem, case_id)
        read_flag, bench_bc_el = bench_bc_log_read(bw_bench_bc_filename)
        if read_flag == 0:
            cont_flag = 1
            break
        #print(bench_bc_el.to_markdown())
        bench_bc_el['src'] = elem
        bench_bc = pd.concat([bench_bc,bench_bc_el])
    if cont_flag:
        continue
    bench_bc.sort_values(inplace = True, by = ['dim1', 'dest', 'src'])
    #print(bench_bc.to_markdown()) 
    max_dim = bench_bc['dim1'].max()
    #print(max_dim)
    bench_bc_gb = bench_bc[bench_bc['dim1'] == max_dim]
    #print(bench_bc_gb.to_markdown())
    print('====================================================================================================')
    print('CHL_MEMLOCS = %d\n' %(CHL_MEMLOCS))
    print('One-directional:')
    for elem in memloc_list:
        for elem1 in memloc_list:
    print('====================================================================================================')

1
[0]
|    |   dest |   dim1 |   dim2 |   elemSize |      bw |   src |
|---:|-------:|-------:|-------:|-----------:|--------:|------:|
|  5 |      0 |   8192 |   8192 |          8 | -1      |     0 |
|  5 |      0 |   8192 |   8192 |          8 | 48.5119 |     1 |
|  5 |      0 |   8192 |   8192 |          8 | 12.3011 |     2 |
|  5 |      0 |   8192 |   8192 |          8 | 12.4131 |     3 |
CHL_MEMLOCS = 4

2
[1]
|    |   dest |   dim1 |   dim2 |   elemSize |      bw |   src |
|---:|-------:|-------:|-------:|-----------:|--------:|------:|
|  5 |      1 |   8192 |   8192 |          8 | 48.5116 |     0 |
|  5 |      1 |   8192 |   8192 |          8 | -1      |     1 |
|  5 |      1 |   8192 |   8192 |          8 | 12.2997 |     2 |
|  5 |      1 |   8192 |   8192 |          8 | 12.4126 |     3 |
CHL_MEMLOCS = 4

3
[0, 1]
|    |   dest |   dim1 |   dim2 |   elemSize |      bw |   src |
|---:|-------:|-------:|-------:|-----------:|--------:|------:|
| 10 |      0 |   8192 |   8192 |  